# **NOTEBOOK D'UTILISATION DE CERTAINES FONCTIONALITEES PERSO**

In [2]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\martin.schoreisz\git\Lin_uniq_BdxM\Lin_uniq_BdxM\Traitements')
import Connexion_Transfert as ct
from datetime import datetime
from collections import Counter
from geoalchemy2 import Geometry,WKTElement
import geopandas as gp
import pandas as pd
import numpy as np
import shutil

from Outils import creer_graph, plus_proche_voisin,nb_noeud_unique_troncon_continu,verif_index, gp_changer_nom_geom
from Decorateurs import concat_df

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_colwidth', 50)

In [ ]:
#TELECHARGER DES DONNEES depuis leserveur geomatique et les passer dans un fichier
@concat_df
def creer_dept(fichier):
    return gp.read_file(fichier)

liste_f=[fr'D:\temp\PlaMADE\donnees\N_TRONCON_ROUTE_BDT_0{d}.SHP' for d in  ['16','17','86','79','19','23','87','33','24','40','64','47']]
df_tot=creer_dept(liste_f)

In [ ]:
#trsnfere des données dans une bdd
with ct.ConnexionBdd('local_otv_station_gti') as c :
    for d in ['17','86','79','19','23','87','33','24','40','64','47'] :
        fichier=fr'D:\temp\PlaMADE\donnees\N_TRONCON_ROUTE_BDT_0{d}.SHP'
        ct.ogr2ogr_shp2pg(c.connstringOgr,fichier,schema='plamade', table='troncon_route_bdt17_l',SRID='2154',
                          geotype='MULTILINESTRINGZ', dims=3, creationMode='-append -update',encodageClient='UTF-8')

In [6]:
with ct.ConnexionBdd('local_otv') as c :
    fichier=r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\IGN\Bdtopo_BdxMet_ed18_l.shp'
    ct.ogr2ogr_shp2pg(c.connstringOgr,fichier,schema='linearisation_bm', table='Bdtopo_BdxMet_ed18_l',SRID='2154',
                          geotype='LINESTRINGZ', dims=3, creationMode='',encodageClient='UTF-8')

debut import fichier D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\IGN\Bdtopo_BdxMet_ed18_l.shp avec shape2pg à 10:55:40 
 avec commande ogr2ogr  -f "postgreSQL" --config PG_USE_COPY YES -a_srs "EPSG:2154"  -nlt LINESTRINGZ -dim 3 -lco "SCHEMA=linearisation_bm" -lco GEOMETRY_NAME=geom PG:"host=localhost dbname=otv user=postgres password=postgres port=5432" D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\IGN\Bdtopo_BdxMet_ed18_l.shp -nln linearisation_bm.Bdtopo_BdxMet_ed18_l 
Fait


In [5]:
with ct.ConnexionBdd('local_otv_station_gti') as c :
    fichier=fr'D:\temp\PlaMADE\donnees\N_TRONCON_ROUTE_BDT_016.SHP'
    ct.Ogr2ogr_pg2shp(c.connstringOgr,r'D:\temp\otv\2018\livrables_OTR_2018\sigena_d_2014_2018_p.shp',
                     "select * from livrable.sigena_d_2014_2018_p")

debut export fichier D:\temp\otv\2018\livrables_OTR_2018\sigena_d_2014_2018_p.shp 
 a 14:04:30 
 avec commande ogr2ogr -f "ESRI shapefile" D:\temp\otv\2018\livrables_OTR_2018\sigena_d_2014_2018_p.shp PG:"host=localhost dbname=statio_gti_otv user=postgres password=postgres port=5432"  -sql "select * from livrable.sigena_d_2014_2018_p" 
Fait


In [17]:
pd.read_excel(r'D:\temp\otv\2019\Donnees_source\Grand Dax\wetransfer-d4c091\Donneess_comptage\Comptage G.Chaulet - DAX - 2016\1CHA01.IFX',sheet_name='Tableau 1')

,Unnamed: 0,TABLEAU HORAIRE DE SYNTHESE DES RESULTATS JOURNALIERS,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,Document :,1CHA01.IFX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,Commanditaire :,GRAND DAX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,Service :,Service Voirie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,Voirie :,Voie Communale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Numéro compteur :,1,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,Nombre de voie :,Voie Unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Type :,2,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,Département :,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Indice :,1,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,Localisation :,DAX BOULEVARD GEORGE CHAULET SENS SUD VERS NORD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Section :,1,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,Sens :,Sens 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mode :,4 - VL/PL Vit.,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,Début :,jeudi 04 févr 2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Séquence :,60,NaN,NaN,NaN,NaN
